<a href="https://colab.research.google.com/github/markwang0/GeoFlood/blob/WIP-pygeoflood-class/examples/geoflood_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install in-development branch of GeoFlood


In [3]:
%%capture

inColab = 'google.colab' in str(get_ipython())
if inColab:
    %pip install git+https://github.com/markwang0/geoflood@WIP-pygeoflood-class#egg=pygeoflood
else:
    %pip install -e ..
    pass


### Run GeoFlood operations


In [2]:
# library imports

from pathlib import Path
from pygeoflood import PyGeoFlood

In [3]:
# instantiate PyGeoFlood model class
pgf = PyGeoFlood()

In [4]:
# download input data for demo project and set paths

# set data directory
data_dir = Path("data")
if not data_dir.is_dir():
    data_dir.mkdir()

# DEM from USGS 3DEP
pgf.dem_path = Path(data_dir, "OC1mTest.tif")
# download DEM if it doesn't exist
if not pgf.dem_path.is_file():
    !curl https://utexas.box.com/shared/static/id0in7w8yn38o59sexa50aqunsr7mzk0.tif -Lso data/OC1mTest.tif

# flowline from NFIE (NHD MR) dataset
# https://www.arcgis.com/home/webmap/viewer.html?webmap=9766a82973b34f18b43dafa20c5ef535
pgf.flowline_path = Path(data_dir, "Flowline.shp")
if not pgf.flowline_path.is_file():
    !curl -sL https://utexas.box.com/shared/static/s1vlnymonbidnhul52oja7s83ky1212j.tgz | tar -xz -C $data_dir

# catchment from NFIE (NHD MR) dataset
pgf.catchment_path = Path(data_dir, "Catchment.shp")
if not pgf.catchment_path.is_file():
    !curl -sL https://utexas.box.com/shared/static/jzawwvmb2tjl4yqtnitgvfjcc4ho4tg5.tgz | tar -xz -C $data_dir

# pgf.streamflow_forecast_path = "data/prod_nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc"
# pgf.custom_flowline_path = Path(data_dir, "NHDPlus_H_1209_HU4_GDB.gdb")


In [5]:
# apply nonlinear filter to DEM
pgf.apply_nonlinear_filter()

Computing slope of raw DTM
DEM slope array shape: (3378, 4750)
Computing lambda = q-q-based nonlinear filtering threshold
DEM smoothing Quantile: 0.9
Edge Threshold Value: 0.34356024861335754
Filtered DEM written to data/OC1mTest_filtered.tif

apply_nonlinear_filter completed in 9.7446 seconds


In [6]:
# calculate slope
pgf.calculate_slope()

 slope statistics
 min angle: 0.0
 max angle: 54.863846043721324
 mean slope: 0.13960592
 stdev slope: 0.15432632
Slope raster written to data/OC1mTest_slope.tif

calculate_slope completed in 1.1899 seconds


In [7]:
# calculate curvature
pgf.calculate_curvature()

 using geometric curvature
 curvature statistics
 non-nan curvature cell number: 16045500
 non-nan finite curvature cell number:  mean:  -0.00031314598
 standard deviation:  0.112147875
Curvature raster written to data/OC1mTest_curvature.tif

calculate_curvature completed in 1.1393 seconds


In [8]:
# fill depressions on filtered DEM
pgf.fill_depressions()

Filled DEM written to data/OC1mTest_filled.tif

fill_depressions completed in 7.5472 seconds


In [9]:
# calculate MFD flow accumulation
pgf.calculate_mfd_flow_accumulation()

MFD flow accumulation raster written to data/OC1mTest_mfd_fac.tif

calculate_mfd_flow_accumulation completed in 4.4427 seconds


In [10]:
# calculate D8 flow directions
pgf.calculate_d8_flow_direction()

D8 flow direction raster written to data/OC1mTest_d8_fdr.tif

calculate_d8_flow_direction completed in 1.6855 seconds


In [11]:
# calculate outlets
pgf.find_outlets()

Outlets raster written to data/OC1mTest_outlets.tif

find_outlets completed in 0.2276 seconds


In [12]:
# delineate basins
pgf.delineate_basins()

Basins raster written to data/OC1mTest_basins.tif

delineate_basins completed in 0.9451 seconds


In [13]:
# define skeletons
pgf.define_skeleton(
    write_curvature_skeleton=True,
    write_flow_skeleton=True,
)

Curvature mean:  -0.00031413726
Curvature standard deviation:  0.11232527
Curvature Projection: EPSG:6343
Mean upstream flow:  2617.3564
Flow skeleton written to data/OC1mTest_flow_skeleton.tif
Curvature skeleton written to data/OC1mTest_curvature_skeleton.tif
Combined skeleton written to data/OC1mTest_combined_skeleton.tif

define_skeleton completed in 1.1482 seconds


In [14]:
# geodesic distance (not needed for GeoFlood workflow)
pgf.calculate_geodesic_distance(write_cost_function=True)

Performing fast marching
FMM iteration 1/32: RAM usage: 11.56/16.00 GB (72.3%)
FMM iteration 2/32: RAM usage: 11.49/16.00 GB (71.8%)
FMM iteration 3/32: RAM usage: 11.42/16.00 GB (71.4%)
FMM iteration 4/32: RAM usage: 11.42/16.00 GB (71.4%)
FMM iteration 5/32: RAM usage: 11.39/16.00 GB (71.2%)
FMM iteration 6/32: RAM usage: 11.39/16.00 GB (71.2%)
FMM iteration 7/32: RAM usage: 11.38/16.00 GB (71.1%)
FMM iteration 8/32: RAM usage: 11.39/16.00 GB (71.2%)
FMM iteration 9/32: RAM usage: 11.39/16.00 GB (71.2%)
FMM iteration 10/32: RAM usage: 11.39/16.00 GB (71.2%)
FMM iteration 11/32: RAM usage: 11.39/16.00 GB (71.2%)
FMM iteration 12/32: RAM usage: 11.41/16.00 GB (71.3%)
FMM iteration 13/32: RAM usage: 11.48/16.00 GB (71.7%)
FMM iteration 14/32: RAM usage: 11.47/16.00 GB (71.7%)
FMM iteration 15/32: RAM usage: 11.47/16.00 GB (71.7%)
FMM iteration 16/32: RAM usage: 11.47/16.00 GB (71.7%)
FMM iteration 17/32: RAM usage: 11.37/16.00 GB (71.1%)
FMM iteration 18/32: RAM usage: 11.36/16.00 GB (7

In [15]:
# get channel heads (not needed for GeoFlood workflow)
pgf.identify_channel_heads()

Locating skeleton end points
Counting the number of elements of each connected component
Skeleton region size threshold: 258.7368421052632
Continuing to locate skeleton endpoints
Channel heads shapefile written to data/OC1mTest_channel_heads.shp

identify_channel_heads completed in 7.2085 seconds


In [16]:
# get flowline endpoints
pgf.find_endpoints()

Endpoints csv written to data/OC1mTest_endpoints.csv

find_endpoints completed in 0.0238 seconds


In [17]:
# get binary HAND array
pgf.calculate_binary_hand()

Binary HAND raster written to data/OC1mTest_binary_hand.tif

calculate_binary_hand completed in 3.8633 seconds


In [18]:
# # get custom flowline from NHD HR dataset (optional)
# pgf.rasterize_custom_flowline(layer="NHDFlowline")

In [19]:
# extract channel network
pgf.extract_channel_network()

Cost function weights:
curvature          0.2520 (mean flow accumulation)
mfd_fac            1.0000
binary_hand        0.7500
custom_flowline    0.0000

Cost min: 0.5028838188775905
Cost max: inf
cost shape: (3378, 4750)
Creating path 1/1: RAM usage: 11.53/16.00 GB (72.0%)
Number of endpoints: 1
Channel network raster written to data/OC1mTest_channel_network.tif
Channel network vector written to data/OC1mTest_channel_network.shp

extract_channel_network completed in 1.9566 seconds


In [20]:
# calculate HAND
pgf.calculate_hand()

HAND raster written to data/OC1mTest_HAND.tif

calculate_hand completed in 4.7995 seconds


In [21]:
# segment flowline
pgf.segment_channel_network()

Segmented channel network written to data/OC1mTest_segmented_channel_network.shp

segment_channel_network completed in 0.7987 seconds


In [22]:
# delineate channel network segment catchments
pgf.delineate_segment_catchments()

Segment catchments written to data/OC1mTest_segment_catchments.tif

delineate_segment_catchments completed in 1.4458 seconds


In [23]:
# calculate synthetic rating curves table
pgf.calculate_src()

River attributes written to data/OC1mTest_river_attributes.csv
Synthetic rating curves written to data/OC1mTest_src.csv

calculate_src completed in 5.1691 seconds


In [1]:
# calculate flood stage from synthetic rating curves
# pgf.streamflow_forecast_path must be set or custom_Q must be provided
pgf.calculate_flood_stage(custom_Q=500)

Applying custom streamflow to each segment: 500 cms
Flood stages written to data/OC1mTest_flood_stage.csv

calculate_flood_stage completed in 0.0058 seconds


In [ ]:
pgf.to_paths('data/paths.txt')